In [1]:
import pandas as pd
import pyopenms as oms
import json 
import time
import os
from mass_trace import run_mass_trace_detection
from elution_peak import run_elution_peak_detection
from feature_map import run_feature_mapping
from align_chromatograms import align_featureXML_files
from link_features import link_features
from hmdb_indexing import parse_hmdb_to_dataframe_streaming, consensus_to_feature_dicts, load_kegg_compounds_csv
from batch_correction import normalize_feature_maps
from quality_control import run_qc_advanced
from final_report import run_final_report
from adduct_isotope_annotation import annotate_adducts_isotopes

In [2]:
### FUNCTION TO GET THE STEPS OF THE JSON PIPELINE
def get_pipeline_steps(pipeline_json):
    mzml_file = []
    steps = pipeline_json.get('pipeline', {}).get('steps', [])
    for step in steps:
        if step.get('name') == 'select_mzml_file':
            mzml_files = step.get('parameters', {}).get('mzml_files', [])
    return steps, mzml_files

In [3]:
METABOLOMICS_BASE_PATH = "/media/datastorage/it_cast/metabolomica/test/"
with open('pipeline.json', 'r') as f:
    pipeline_json = json.load(f)
    print(pipeline_json)
    
steps, mzml_files = get_pipeline_steps(pipeline_json)
print("MZML FILES:", mzml_files)
print("STEPS:", steps)

{'pipeline': {'steps': [{'name': 'select_mzml_file', 'parameters': {'mzml_files': ['/media/datastorage/it_cast/metabolomica/test/CC1.mzML', '/media/datastorage/it_cast/metabolomica/test/CC2.mzML', '/media/datastorage/it_cast/metabolomica/test/CC3.mzML']}}, {'name': 'mass_trace_detection', 'parameters': {'polarity': 'negative'}}, {'name': 'elution_peak_detection'}, {'name': 'feature_mapping'}, {'name': 'align_chromatograms', 'parameters': {'normalization_method': 'tic'}}, {'name': 'feature_linking'}, {'name': 'quality_control', 'parameters': {'qc_name_pattern': 'QC', 'save_pdf': True, 'save_csv': True, 'save_json': True, 'output_dir': '/media/datastorage/it_cast/omnis_microservice_db/metabolomics/qc_results'}}, {'name': 'hmdb_search'}, {'name': 'final_report', 'parameters': {'output_dir': '/media/datastorage/it_cast/omnis_microservice_db/metabolomics/final_report'}}]}}
MZML FILES: ['/media/datastorage/it_cast/metabolomica/test/CC1.mzML', '/media/datastorage/it_cast/metabolomica/test/CC2

In [4]:
#### MASS TRACE DETECTION ####
results = {}
# initialize results for each file
for mzml_file in mzml_files:
    results[mzml_file] = {}

for mzml_file in mzml_files:
    exp=oms.MSExperiment()
    oms.MzMLFile().load(mzml_file, exp)
    ms2_count = sum(1 for spec in exp if spec.getMSLevel() == 2)
    print(f"{mzml_file}: MS2 Spectra found: {ms2_count}")

## iterate over steps
for step in steps:
    
    for mzml_file in mzml_files:
        if step.get('name') == 'mass_trace_detection':
            polarity = step.get('parameters', {}).get('polarity', 'negative')
            results[mzml_file]['mass_traces'] = run_mass_trace_detection(mzml_file, polarity)
        elif step.get('name') == 'elution_peak_detection':
            results[mzml_file]['elution_peaks'] = run_elution_peak_detection(results[mzml_file]['mass_traces'])
        elif step.get('name') == 'feature_mapping':
            results[mzml_file]['output_feature_mapping'] = run_feature_mapping(mzml_file, results[mzml_file]['elution_peaks'])
            print(results[mzml_file]['output_feature_mapping'])

            
            

/media/datastorage/it_cast/metabolomica/test/CC1.mzML: MS2 Spectra found: 9532
/media/datastorage/it_cast/metabolomica/test/CC2.mzML: MS2 Spectra found: 9532
/media/datastorage/it_cast/metabolomica/test/CC3.mzML: MS2 Spectra found: 9528
Filtered to negative mode: 11919 spectra out of 11919
Progress of 'mass trace detection':
-- done [took 2.79 s (CPU), 2.86 s (Wall)] -- 
Number of mass traces: 22251
Filtered to negative mode: 11919 spectra out of 11919
Progress of 'mass trace detection':
-- done [took 2.75 s (CPU), 2.81 s (Wall)] -- 
Number of mass traces: 21743
Filtered to negative mode: 11914 spectra out of 11914
Progress of 'mass trace detection':
-- done [took 2.60 s (CPU), 2.68 s (Wall)] -- 
Number of mass traces: 21114
Progress of 'elution peak detection':
-- done [took 1.59 s (CPU), 1.65 s (Wall)] -- 
Number of mass traces after elution peak detection: 19500
Progress of 'elution peak detection':
-- done [took 1.60 s (CPU), 1.64 s (Wall)] -- 
Number of mass traces after elution p

In [5]:
# ALIGNMENT #
feature_files = [results[mzml_file]['output_feature_mapping'] for mzml_file in mzml_files]
metrics = align_featureXML_files(feature_files)
ref_sample = None
normalization_method = "tic"
ref_index = feature_files.index(ref_sample) if ref_sample and ref_sample in feature_files else 0
normalized_files = normalize_feature_maps(feature_files, ref_index, normalization_method)
for i, mzml_file in enumerate(mzml_files):
    results[mzml_file]['output_feature_mapping'] = normalized_files[i]
    


Number of features in /media/datastorage/it_cast/omnis_microservice_db/metabolomics/CC1_feature_map.featureXML: 15733
Number of features in /media/datastorage/it_cast/omnis_microservice_db/metabolomics/CC2_feature_map.featureXML: 15057
Number of features in /media/datastorage/it_cast/omnis_microservice_db/metabolomics/CC3_feature_map.featureXML: 14292
Reference map index: 2
Saving aligned feature map to /media/datastorage/it_cast/omnis_microservice_db/metabolomics/aligned_0.featureXML
Saving aligned feature map to /media/datastorage/it_cast/omnis_microservice_db/metabolomics/aligned_1.featureXML
Saving reference feature map to /media/datastorage/it_cast/omnis_microservice_db/metabolomics/aligned_2.featureXML

Alignment metrics:
File: /media/datastorage/it_cast/omnis_microservice_db/metabolomics/CC1_feature_map.featureXML
  Features: 15733
  RT mean before: 330.40, after: 316.58
  RT std before: 191.40, after: 181.79
File: /media/datastorage/it_cast/omnis_microservice_db/metabolomics/CC

In [6]:
# FEATURE LINKING #
feature_files = [results[mzml_file]['output_feature_mapping'] for mzml_file in mzml_files]
output_file_consensus= os.path.join(METABOLOMICS_BASE_PATH, 'final_consensusXML')
results[mzml_file]["linked_features"] = link_features(feature_files, output_file_consensus)
results["consensus_file"] = output_file_consensus

Progress of 'computing RT transformations':
-- done [took 0.16 s (CPU), 0.19 s (Wall)] -- 
Progress of 'linking features':
-- done [took 0.32 s (CPU), 0.33 s (Wall)] -- 
ConsensusMap contains 45082 invalid references to maps:
  wrong id=0 (occurred 15733x)
  wrong id=1 (occurred 15057x)
  wrong id=2 (occurred 14292x)

ConsensusXMLFile::store():  found 27097 invalid unique ids
Consensus map saved to /media/datastorage/it_cast/metabolomica/test/final_consensusXML


The ConsensusXML file contains invalid maps or references thereof. Please fix the file or notify the maintainer of this tool if you did not provide a consensusXML file! Note that this warning will be a fatal error in the next version of OpenMS!


In [7]:
# ADDUCT ISOTOPE ANNOTATION
polarity = "negative"
user_adducts = None
user_charge = None

for i, mzml_file in enumerate(mzml_files):
    input_fxml = feature_files[i]
    output_fxml = input_fxml.replace(".featureXML", "_adduct_annotated.featureXML")
    try:
        annotate_adducts_isotopes(input_fxml, output_fxml, polarity)
        results[mzml_file]['output_feature_mapping_adduct_annotation'] = output_fxml
    except Exception as e:
        print(f"Error in adduct annotation for {mzml_file}: {str(e)}")
        results[mzml_file]['output_feature_mapping'] = input_fxml

Adding neutral: ---------- Adduct -----------------
Charge: 0
Amount: 1
MassSingle: 215.581
Formula: C1H202
log P: -0.693147

MassExplainer table size: 14
Error in adduct annotation for /media/datastorage/it_cast/metabolomica/test/CC1.mzML: MetaboliteFeatureDeconvolution computation failed: WARNING!!! implicit number of default adduct is negative!!! left:-2 right: -2

Adding neutral: ---------- Adduct -----------------
Charge: 0
Amount: 1
MassSingle: 215.581
Formula: C1H202
log P: -0.693147

MassExplainer table size: 14
Error in adduct annotation for /media/datastorage/it_cast/metabolomica/test/CC2.mzML: MetaboliteFeatureDeconvolution computation failed: the value '1 -3' was used but is not valid; feature charge and putative positive mode charge switch charge direction!
Adding neutral: ---------- Adduct -----------------
Charge: 0
Amount: 1
MassSingle: 215.581
Formula: C1H202
log P: -0.693147

MassExplainer table size: 14
Error in adduct annotation for /media/datastorage/it_cast/metabo

In [8]:
xml_db_path = '/media/datastorage/it_cast/omnis_microservice_db/tools/hmdb_metabolites.xml'
hmdb_df = parse_hmdb_to_dataframe_streaming(xml_db_path)
kegg_df = load_kegg_compounds_csv('/media/datastorage/it_cast/omnis_microservice_db/tools/kegg_compounds.csv')
output_file_consensus = results.get("consensus_file")
if output_file_consensus:
    feature_dicts = consensus_to_feature_dicts(output_file_consensus, hmdb_df, kegg_df, ppm=10)
                
    df = pd.DataFrame(feature_dicts)
    df = df[df['hmdb_matches'].apply(lambda x: len(x) > 0)]
    df['hmdb_matches'] = df['hmdb_matches'].apply(lambda x: x[0] if x else {})
    df['hmdb_matches'] = df['hmdb_matches'].apply(lambda x: x['name'] if isinstance(x, dict) else x)
    df = df.sort_values(by='input_maps', key=lambda x: x.str.len(), ascending=False).drop_duplicates(subset=['hmdb_matches'], keep='first')
            
    output_csv_path = os.path.join(METABOLOMICS_BASE_PATH, "hmdb_search_results.csv")
    df.to_csv(output_csv_path, index=False)
    results[mzml_file]["hmdb_search_results"] = output_csv_path
    print(f"HMDB search results saved to {output_csv_path}")
else:
    print("Consensus file not found, skipping HMDB search*")

ConsensusMap contains 45082 invalid references to maps:
  wrong id=0 (occurred 15733x)
  wrong id=1 (occurred 15057x)
  wrong id=2 (occurred 14292x)

Processed 27096 features from consensus XML.
kegg compounds: 19508,  shape: (5, 4)
┌─────────┬──────────────────┬────────────────────────┬────────────┐
│ KEGG_ID ┆ Name             ┆ Formula                ┆ Exact_Mass │
│ ---     ┆ ---              ┆ ---                    ┆ ---        │
│ str     ┆ str              ┆ str                    ┆ f64        │
╞═════════╪══════════════════╪════════════════════════╪════════════╡
│ C00012  ┆ Peptide          ┆ C2H4NO2R(C2H2NOR)n     ┆ null       │
│ C00017  ┆ Protein          ┆ C2H4NO2R(C2H2NOR)n     ┆ null       │
│ C00028  ┆ Acceptor         ┆ null                   ┆ null       │
│ C00030  ┆ Reduced acceptor ┆ null                   ┆ null       │
│ C00039  ┆ DNA              ┆ C10H17O8PR2(C5H8O5PR)n ┆ null       │
└─────────┴──────────────────┴────────────────────────┴────────────┘ 


HMDB 

In [9]:
print(df.head(20))

            mz       rt                                       hmdb_matches  \
16    292.1402   1.0677                             Phenylbutyrylglutamine   
15    493.3172   5.3310                               LysoPC(16:1(9Z)/0:0)   
14    403.1398   3.8242  (2S,3S)-3-[[3,5-Bis(trifluoromethyl)phenyl]met...   
6013  308.1372   5.0108  2-((2-(Dimethylamino)ethyl)thio)-3-phenylquino...   
6014  285.2434   8.9294                                Dehydroabietylamine   
6015  431.1924   8.8770                                          Fusarin C   
166   831.4983   9.6826       PS(18:2(9Z,12Z)/22:6(4Z,7Z,10Z,13Z,16Z,19Z))   
173   340.1874   3.8354                         Glycyl-l-histidyl-l-lysine   
174   337.1617   0.8651                          Furylacryloylalanyllysine   
141   486.2569   2.7906                      Prednisolone valerate acetate   
142   428.1744   7.0272                                       Miricorilant   
144   361.1293   3.8350                           Pyriminobac-me

In [10]:
df_comparing = pd.read_csv("m_MTBLS12772_LC-MS_negative_reverse-phase_metabolite_profiling_v2_maf.tsv", sep='\t')

In [11]:
# Extract unique metabolite names from MAF (using df_comparing)
hmdb_names_maf = set(df_comparing['metabolite_identification'].dropna().unique())

# Extract unique hmdb_matches from pipeline results (using df)
pipeline_names = set(df['hmdb_matches'].dropna().unique())

# Find intersection: how many MAF names are in the pipeline's hmdb_matches
intersection = hmdb_names_maf & pipeline_names

print(f"Total unique metabolites in MAF: {len(hmdb_names_maf)}")
print(f"Total unique hmdb_matches in pipeline: {len(pipeline_names)}")
print(f"Number of matching metabolites (MAF names in pipeline hmdb_matches): {len(intersection)}")
print(f"Matching metabolites: {sorted(intersection)}")

Total unique metabolites in MAF: 239
Total unique hmdb_matches in pipeline: 5403
Number of matching metabolites (MAF names in pipeline hmdb_matches): 26
Matching metabolites: ['1-(11Z-eicosenoyl)-glycero-3-phosphate', '2-Pentanamido-3-phenylpropanoic acid', '3b,12a-Dihydroxy-5a-cholanoic acid', '4-Hydroxyphenylacetylglutamic acid', 'APGPR Enterostatin', 'CPA(16:0/0:0)', 'Ciclesonide', 'Clocinizine', 'Collettiside I', 'Dehydroxymethylflazine', 'Dimethyl 3-methoxy-4-oxo-5-(8,11,14-pentadecatrienyl)-2-hexenedioate', 'Dolichosterone', 'Ganoderic acid F', 'Ganoderic acid L', 'Jasmonic acid', 'Metabolite M6', 'PE(14:0/20:2(11Z,14Z))', 'PG(16:0/18:2(9Z,12Z))', 'PS(16:0/20:3(8Z,11Z,14Z))', 'Palmitoyl glucuronide', 'Pangamic acid', 'Pentadecanoylglycine', 'Pitheduloside I', 'Rigin', 'TG(20:5(5Z,8Z,11Z,14Z,17Z)/18:3(9Z,12Z,15Z)/20:5(5Z,8Z,11Z,14Z,17Z))', 'Tetrahydrodeoxycorticosterone']
